# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = '/home/robin/GIT/ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
print(DATA_FOLDER)

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
import calendar
import glob
from datetime import datetime

In [ ]:
# Write your answer here

# Method to concat all the reports files for each country in a single dataframe

def concat_files(path,date_str):
    the_files = glob.glob(path +"/*.csv")
    data_frame = pd.DataFrame()
    for x in the_files:
        df_temp = pd.read_csv(x,parse_dates=[date_str])
        data_frame = pd.concat([data_frame,df_temp])
    return data_frame

# Todo: remove method select_rows for better readability
# Method to select all the rows matching the 'wanted' argument
def select_rows(df,reference,wanted):
    return df.loc[df[reference] == wanted]

# Method to add a Month column
def add_month(df):
    copy_df = df.copy()
    months = [calendar.month_name[x.month] for x in copy_df.Date]
    copy_df['Month'] = months
    return copy_df

# Method to calculate the average number of new cases per month
def calculate_avg_new_cases(df,months):
    avg_cases = np.zeros(len(months))
    for i in range(len(months)):
        temp = select_rows(df,'Month',months[i]).Total_new_cases.values.astype(float) 
        if(len(temp) < 2 ): # if less than two points, assign NaN
            avg_cases[i] = np.nan
        else: # Otherwise sum/number
            avg_cases[i] = temp.sum()/temp.shape[0]
        if avg_cases[i] < 0:
            raise ValueError('Value of an average is less than 0')
    return avg_cases  

# Method to calculate the average number of deaths per month
def calculate_avg_new_deaths(df,months):
    avg_cases = np.zeros(len(months))
    for i in range(len(months)):
        temp = select_rows(df,'Month',months[i]) # selecting relevant month
        temp = temp.reset_index()
        delta_t = temp['Date'].loc[len(temp)-1] - temp['Date'].loc[0] # calculate time-span of data
        temp = temp.Total_new_deaths.values.astype(float) # extracting array of floats
        if(temp.shape[0]<2):
            avg_cases[i] = np.nan
        elif(temp[len(temp)-1]-temp[0] > 0):   # if there is a difference between 1st/last time point
            avg_cases[i] = (temp[len(temp)-1]-temp[0])/delta_t.days
        else: 
            avg_cases[i] = (temp[len(temp)-2]-temp[0])/delta_t.days
        if avg_cases[i] < 0:
            raise ValueError('Value of an average is less than 0')
    return avg_cases   

# Method to handle Sierra Leon and Liberia data
def handle_data(df,desc1,desc2,desc3,new_total):
    sl_1 = select_rows(df,'Description',desc1)
    sl_2 = select_rows(df,'Description',desc2)
    sl_3 = select_rows(df,'Description',desc3)
    new_cases_registered =  [int(a) + int(b) + int(c) for a,b,c in zip(sl_1.Totals,sl_2.Totals,sl_3.Totals)]
    sl_1[new_total] = new_cases_registered
    sl_1 = sl_1[['Date',new_total]]
    #sl_1 = add_month(sl_1,'Date')
    return sl_1

In [ ]:
# Create Guinea DF
guinea_folder = DATA_FOLDER + '/ebola/guinea_data/'
g_df = concat_files(guinea_folder,'Date')
g_df.sort_values('Date', inplace=True)
print('Shape of Guinea DF:', g_df.shape)

# We can check all unique descriptions in order to choose what 
# to base our calculations on as well as to be sure not to miss anything
g_df['Description'].unique()

# We can then see which descriptors appear to be the most complete,
# in this case it seems using one of the total deaths descriptors
# is the only sensible option, the new deaths data sets are much
# less complete. We'll take the confirmed+probables+suspects
# in order to be most complete, so the end results are probably
# overestimating the deaths a bit
print(g_df.loc[g_df['Description']=='Total deaths of probables'].shape)
print(g_df.loc[g_df['Description']=='Total deaths of confirmed'].shape)
print(g_df.loc[g_df['Description']=='Total deaths (confirmed + probables + suspects)'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today (probables)'].shape)
print(g_df.loc[g_df['Description']=='New deaths registered today (suspects)'].shape)
g_df.loc[g_df['Description']=='Total deaths (confirmed + probables + suspects)'][['Date','Description','Totals']]

In [ ]:
# We only keep the columns of interest
g_df = g_df[['Date', 'Description', 'Totals']]

# We select the rows that we need in order to calculate our values
g_new_cases = select_rows(g_df,'Description','Total new cases registered so far')
g_new_deaths = select_rows(g_df,'Description','Total deaths (confirmed + probables + suspects)')
g_new_deaths.head()

In [ ]:
# Handling the new Dataframes
g_new_cases = g_new_cases[['Date','Totals']]
g_new_cases.columns = ['Date','Total_new_cases']

g_new_deaths = g_new_deaths[['Date','Totals']]
g_new_deaths.columns = ['Date','Total_new_deaths']

# We merge the 2 of them to obtain the new cases and the deaths for Guinea
guinea_df = pd.merge(g_new_cases,g_new_deaths)
guinea_df['Country'] = 'Guinea'
guinea_df = guinea_df[['Country','Date','Total_new_cases','Total_new_deaths']]

In [ ]:
g_months = add_month(guinea_df)
g_months.head(10)

In [ ]:
temp = select_rows(g_months,'Month','September')
temp

We finally get to calculate average daily new cases and deaths per month.
In this data, we assume that the *Total deaths (confirmed + probables + suspects)* row we used from the original data is a cumulative value of the deaths.
However, the case for the *Total new cases registered so far* column was not as clear. The values for August appear to increasee monotonically as a a cumulative variable would. While the values for September seem to fluctuate more. We therefore chose to interpret this column as a *daily* new cases registered value, as this is compatible with the fluctuations observed in September.

In [ ]:
# Calculating the daily average per month of new cases and deaths for Guinea
months = ['August','September','October']
g_results_deaths = calculate_avg_new_deaths(g_months,months)
print(g_results_deaths)
g_results_cases = calculate_avg_new_cases(g_months,months)
g_results_cases

# NaN values is for the months were we dont have enough data to make an average,
# in this case there is only one data point for October, so we chose to exclude it

In [ ]:
# Create the Sierra Leone DataFrame
sl_folder = DATA_FOLDER + '/ebola/sl_data/'
sl_df = concat_files(sl_folder,'date')
sl_df.sort_values('date', inplace=True)
sl_df[['date' , 'variable' , 'National']]

# We only keep the columns of interest 
sl_df = sl_df[['date','variable','National']]
sl_df.columns = ['Date' , 'Description' , 'Totals']
sl_df = sl_df.fillna(0)

# Creating dataframes for new cases and new deaths
sl_new_cases = handle_data(sl_df,'new_confirmed','new_probable','new_suspected','Total_new_cases')
sl_new_cases1 = sl_new_cases.loc[sl_new_cases['Total_new_cases'] > 0]
sl_new_deaths = handle_data(sl_df,'death_confirmed','death_probable','death_suspected','Total_new_deaths')
sl_new_deaths1 = sl_new_deaths.loc[sl_new_deaths['Total_new_deaths'] > 0]

# Forming the Sierra Leon DataFrame
sierra_leon_df = pd.merge(sl_new_cases,sl_new_deaths)
sierra_leon_df['Country'] = 'Sierra Leone'
sl_df

In [ ]:
# Calculating the daily average per month of new cases and deaths for Sierra Leon
months = ['August','September','October','November','December']
sl_results_deaths = calculate_avg_new_deaths(add_month(sl_new_deaths1,),months)
sl_results_deaths
sl_results_cases = calculate_avg_new_cases(add_month(sl_new_cases1,),months)
sl_results_cases
sl_results_deaths

In [ ]:
# Create the Liberia DataFrame
lib_folder = DATA_FOLDER + '/ebola/liberia_data/'
lib_df = concat_files(lib_folder,'Date')
lib_df.sort_values('Date', inplace=True)

# We only keep the columns of interest 
lib_df = lib_df[['Date','Variable','National']]
lib_df.columns = ['Date' , 'Description' , 'Totals']
lib_df = lib_df.fillna(0)
lib_df.head(10)

In [ ]:
# Creating dataframes for new cases and new deaths
lib_new_cases = handle_data(lib_df,'New Case/s (Suspected)','New Case/s (Probable)','New Case/s (Probable)','Total_new_cases')
lib_new_cases1 = lib_new_cases.loc[lib_new_cases['Total_new_cases'] > 0]
lib_new_deaths = handle_data(lib_df,'Total death/s in confirmed cases','Total death/s in probable cases','Total death/s in suspected cases','Total_new_deaths')
lib_new_deaths1 = lib_new_deaths.loc[lib_new_deaths['Total_new_deaths'] > 0]
lib_new_cases1
lib_new_deaths1.head()

In [ ]:
# Forming the Sierra Leon DataFrame
liberia_df = pd.merge(lib_new_cases,lib_new_deaths)
liberia_df['Country'] = 'Liberia'

In [ ]:
# Calculating the daily average per month of new cases and deaths for Liberia
months = ['June','July','August','September','October','November','December']
lib_results_deaths = calculate_avg_new_deaths(add_month(lib_new_deaths1,),months)
lib_results_deaths
lib_results_cases = calculate_avg_new_cases(add_month(lib_new_cases1,),months)
lib_results_cases
#lib_results_deaths
#liberia_df

In [ ]:
#Forming the Final DataFrame contaning the relevant information for the 3 different country

df = pd.concat([guinea_df,sierra_leon_df,liberia_df])

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

First we are going to concatenate all excel files into one `DataFrame`. We'll make sure we have a unique index by resetting it to a temporary 0-x numerical index. We're going to be using the variable `xl_lengths` to keep track of the size of the different tables that are concatenated.

In [ ]:
mb_data = pd.DataFrame(columns=['raw','counts'])
nb_excels = 9
xl_lengths = np.zeros(nb_excels, dtype=int)
counts = 0

# Concat all excel files
for i in range(nb_excels):
    mb_new = pd.read_excel(DATA_FOLDER+"/microbiome/MID"+str(i+1)+".xls", header=None, names=['raw','counts'])
    mb_new.fillna('unknown', inplace=True)
    xl_lengths[i] = np.size(mb_new.index)
    counts += mb_new['raw'].size
    mb_data = pd.concat([mb_data, mb_new], axis=0, join='outer')

# Reset indices
mb_data = mb_data.reset_index() # creates a new 'index' column
mb_data = mb_data.drop('index', axis=1) # dropping this new column

print('Nb. of data points:', counts)
print('Data frame shape', mb_data.shape)
mb_data.head()

We read in the metadata :

In [ ]:
mb_metadata = pd.read_excel(DATA_FOLDER+"/microbiome/metadata.xls")
mb_metadata.fillna('unknown', inplace=True)
print('Metadata DF index is unique:',mb_metadata.index.is_unique)
mb_metadata.head(3)

We now initialise the new columns that will be filled in with the metadata information

In [ ]:
mb_data = pd.concat([mb_data, pd.DataFrame(columns=mb_metadata.columns)], axis=1)
print('Data frame shape:',mb_data.shape)
mb_data.head()

We now add the metadata to the corresponding slices of the overall `DataFrame` using our previous `xl_length` variable to keep track of when the imported tables end and another begins.

In [ ]:
idx = 0
for j in range(np.size(xl_lengths)):
    if j >= 1:
        idx = sum(xl_lengths[0:j])
    for i in mb_metadata.columns:
        mb_data[i][idx:idx+xl_lengths[j]+1] = mb_metadata[i][j]
#print(mb_data[mb_metadata.columns])#[0:xl_lengths[0]+10])

print(xl_lengths)
print('Final DataFrame index is unique', mb_data.index.is_unique)
mb_data.head()

We now create a meaningful index using all the metadata that we imported and joined together. We'll also check that the index is unique

In [ ]:
new_index = ['raw', 'BARCODE', 'GROUP', 'SAMPLE']
mb_data = mb_data.set_index(new_index)
print('Index is unique: ', mb_data.index.is_unique)
mb_data.head()

We can now look at all the data concerning one type of organism

In [ ]:
mb_data.loc['Archaea "Crenarchaeota" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera']

Depending on how we wish to view the combined data frame, for example all tissue samples of a specific organism, we can swap indices to make this easier:

In [ ]:
mb_data = mb_data.swaplevel('BARCODE','SAMPLE')
mb_data.sort_index(axis=0, inplace=True) # without sorting a PerformanceWarning is thrown
mb_data.loc['Archaea "Crenarchaeota" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera', 'tissue']

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

In [ ]:
df = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic', header=0)
print('Shape of DataFrame:', df.shape)
print(type(df['sex'][0]))
df.head()

## For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

### Task 3.1 

In [ ]:
# Question: Describe the type and the value range of each attribute. 

# We assume that we can describe the value range of each attribute
# only if it's a numerical type

types = df.dtypes
value_range = df._get_numeric_data().apply(lambda x: (x.min(), x.max()))

types.index.name = 'Attribute'
types.columns = 'Type'

value_range.index.name = 'Attribute'
value_range.columns = 'Range'

res = pd.concat(dict(Range=value_range, Type=types), axis=1).fillna('-')
res

In [ ]:
# Question: Indicate and transform the attributes that can be Categorical.

# By visual inspection we can argue that the following columns could be
# categorical : 'pclass', 'survived', 'sex', 'sibsp', 'parch', 'embarked'
#
# We assumed that for an attribute to be categorical it should have 
# <= 10 distinct elements. (NaN is not considered as a distinct value)
#
# In this case, both these criteria lead to the same selection of
# attributes
#
# We chose to leave out 'cabin' as a category due to the large number
# of values that are present within it, especially compared to other
# columns such as those mentioned above 


#categories = [v for v in df.columns if (pd.Series(['unique']).isin(df[v].describe().index.tolist())).bool()
                                            #and df[v].describe()['unique'] < 10 ]
categories = [v for v in df.columns if np.size(df[v].unique()) < 10 ]

print('Attributes to be made Categorical:', categories)
for i in categories:
    df[i] = df[i].astype('category')
df.head()

In [ ]:
df.head()

### Task 3.2

In [ ]:
# Question: Plot a histogram for the travel class attribute

#xs = [1, 2, 3]
#l = df['pclass'].tolist()
#ys = [l.count(1), l.count(2), l.count(3)]

temp = []
[temp.append(v) for v in df['pclass'].index if not df['pclass'][v] in temp]


hist_plot = df.pclass.value_counts().plot(kind='bar')
hist_plot.set_title('Travel Class')
hist_plot.set_xlabel('Class')
hist_plot.set_ylabel('Passengers')
np.sum(df.pclass.value_counts())


In [ ]:
# Question: Plot a histogram for the embarkation port attribute

nb_ports = df['embarked'].describe()['unique']
df.embarked.unique()

hist_plot = df.embarked.value_counts().plot(kind='bar')
hist_plot.set_title('Embarkation port')
hist_plot.set_xlabel('Port')
hist_plot.set_ylabel('Passengers')
sum(df.embarked.value_counts())

In [ ]:
# Question: Plot a histogram for the sex attribute

hist_plot = df.sex.value_counts().plot(kind='bar')
hist_plot.set_title('Sex')
hist_plot.set_xlabel('Sex')
hist_plot.set_ylabel('Passengers')
np.sum(df.sex.value_counts())

In [ ]:
# Question: Plot a histogram for the age attribute

nb_bins = np.ceil(df.age.max()/10).astype('int')
hist_plot = df.age.hist(bins=nb_bins, grid=False, xlabelsize=11, ylabelsize=11, figsize=(10, 6))
hist_plot.set_title('Age')
hist_plot.set_xlabel('Ages')
hist_plot.set_ylabel('Passengers')

### Task 3.3

In [ ]:
# Question: Calculate the proportion of passengers by cabin floor. 
# Present your results in a pie chart.

# We assumed that the information on the cabin floor is available 
# in the column 'cabin' where the letter indicates the floor.
# the number that follows indicates a specific cabin on that floor.

# Unfortunately, this column is also filled with NaN's values.
# For this statistic we won't try guessing the cabin floor for 
# passenger where the data is not provided. Our analysis relies
# only on values that are already in the dataset.

tot = df.cabin.describe()['count']

temp = df.cabin.dropna(axis=0).apply(lambda x: str(x)[:1])

D = {}
for v in temp:
    if not v in D:
        D[v] = 1
    else:
        D[v] += 1


labels = list(D.keys())
values = list(D.values())


plt.pie(values, labels=labels)
plt.show()


### Task 3.4

In [ ]:
# Question: For each travel class, calculate the proportion of the
# passengers that survived. Present your results in pie charts.

tot = df.survived.describe()['count']

class_tot = (df.groupby('pclass').survived.describe())['count']
pclass_survived = df.groupby(['pclass', 'survived']).survived.describe()
pclass_survived = (pclass_survived[pclass_survived['top'] == 1])['freq']
pclass_survived = pclass_survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, pclass_survived], axis=1)
prop = stats['freq'] / stats['count']

l = df.set_index('pclass').index.categories
plt.pie(list(prop), labels=list(l))
plt.show()
